# Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

### Заказчику важны:
- качество предсказания;
- скорость предсказания;
- время обучения.


# Инструкция по выполнению проекта

Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: скорость работы, точность результата.
1. Загрузите и подготовьте данные.
2. Обучите разные модели. Для каждой попробуйте различные гиперпараметры.
3. Проанализируйте скорость работы и качество моделей.

### Примечания:

- Для оценки качества моделей применяйте метрику RMSE.
- Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.
- Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найдите её.
- Поскольку модель градиентного бустинга может обучаться долго, измените у неё только два-три параметра.
- Если перестанет работать Jupyter Notebook, удалите лишние переменные оператором del: 'del features_train'

# Описание данных

### Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

### Целевой признак
- Price — цена (евро)

# 1.0 Подготовка данных

In [1]:
#conda install -c conda-forge lightgbm

In [2]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [3]:
# Откроем таблицу двумя путями
server_data = '/datasets/autos.csv'
local_data = '/Users/mmaximmaximovgmail.com/Desktop/DS/project 11/autos.csv'

try:
    auto = pd.read_csv(server_data)
except:
    auto = pd.read_csv(local_data)

In [4]:
display(auto.head())
display(auto.info())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

In [5]:
auto.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

### Мини-вывод:
- Имеются пропуски в колонках: VehicleType, Gearbox, Model, FuelType, NotRepaired
- Все колонки имеют верхний регистр
- Колонки DateCrawled, DateCreated, LastSeen имеют не правильный тип данных

In [6]:
# Сначала поменяем тип данных для колонок с датами

date = ['DateCrawled', 'DateCreated', 'LastSeen']

for q in date:
    auto[q] = pd.to_datetime(auto[q], format='%Y.%m.%dT%H:%M:%S') 
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Kilometer          354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  NotRepaired        283215 non-null  object        
 12  DateCreated        354369 non-null  datetime64[ns]
 13  NumberOfPictures   354369 non-null  int64   

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 👍
</div>

In [7]:
# Теперь изучем какие данных больше в каждой колонке где есть пропуски. Начну с автомобильного кузова
auto['VehicleType'].value_counts().head()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
Name: VehicleType, dtype: int64

In [8]:
auto['VehicleType'] = auto['VehicleType'].fillna('sedan')

In [9]:
# Тип коробки передач
auto['Gearbox'].value_counts().head()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

In [10]:
auto['Gearbox'] = auto['Gearbox'].fillna('manual')

In [11]:
# Модель авто
auto['Model'].value_counts().head()

golf     29232
other    24421
3er      19761
polo     13066
corsa    12570
Name: Model, dtype: int64

In [12]:
auto['Model'] = auto['Model'].fillna('unknown')

In [13]:
# Топливо
auto['FuelType'].value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

In [14]:
auto['FuelType'] = auto['FuelType'].fillna('petrol')

In [15]:
# Была в ремонте?
auto['NotRepaired'].value_counts()

no     247161
yes     36054
Name: NotRepaired, dtype: int64

In [16]:
auto['NotRepaired'] = auto['NotRepaired'].fillna('no')

In [17]:
auto.duplicated().sum()

5

In [18]:
auto = auto.drop_duplicates()

In [19]:
# Убрал дубликаты
auto.duplicated().sum()

0

In [20]:
auto.describe(include='all')

/var/folders/td/gy6t5n6555nbwymks58cvbjr0000gn/T/ipykernel_6308/13986645.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  auto.describe(include='all')


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354364,354364.000000,354364,354364.000000,354364,354364.000000,354364,354364.000000,354364.000000,354364,354364,354364,354364,354364.0,354364.000000,354364
unique,271174,NaN,8,NaN,2,NaN,251,NaN,NaN,7,40,2,109,NaN,NaN,179150
top,2016-03-24 14:49:47,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,NaN,128946,NaN,288081,NaN,29232,NaN,NaN,249244,77012,318311,13718,NaN,NaN,17
first,2016-03-05 14:06:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-10 00:00:00,NaN,NaN,2016-03-05 14:15:08
last,2016-04-07 14:36:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-07 00:00:00,NaN,NaN,2016-04-07 14:58:51
mean,NaN,4416.655608,NaN,2004.234471,NaN,110.093723,NaN,128211.373051,5.714641,NaN,NaN,NaN,NaN,0.0,50508.461698,NaN
std,NaN,4514.159690,NaN,90.228593,NaN,189.851590,NaN,37905.136957,3.726433,NaN,NaN,NaN,NaN,0.0,25783.124276,NaN
min,NaN,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30165.000000,NaN


In [21]:
# Можем позволить себе удалить некоторые колонки, т.к они не пригодятся
auto = auto.drop(['DateCrawled','DateCreated','LastSeen','NumberOfPictures','PostalCode', 'RegistrationMonth'], axis=1)

In [22]:
auto.describe(include='all')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
count,354364.000000,354364,354364.000000,354364,354364.000000,354364,354364.000000,354364,354364,354364
unique,NaN,8,NaN,2,NaN,251,NaN,7,40,2
top,NaN,sedan,NaN,manual,NaN,golf,NaN,petrol,volkswagen,no
freq,NaN,128946,NaN,288081,NaN,29232,NaN,249244,77012,318311
mean,4416.655608,NaN,2004.234471,NaN,110.093723,NaN,128211.373051,NaN,NaN,NaN
std,4514.159690,NaN,90.228593,NaN,189.851590,NaN,37905.136957,NaN,NaN,NaN
min,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,NaN,NaN,NaN
25%,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,NaN,NaN,NaN
50%,2700.000000,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,NaN,NaN,NaN
75%,6400.000000,NaN,2008.000000,NaN,143.000000,NaN,150000.000000,NaN,NaN,NaN


In [23]:
auto = auto.query('RegistrationYear < 2017')
auto = auto.query('40 < Power < 1001')
auto = auto.query('100 < Price < 20001')

### Мини-вывод по 1.0:
- Заполнил все пропуски где нуждалось
- Проверил на правильность написание каждой колонки
- Удалил дубликаты
- Удалил не нужные колонки
- В колонке год регистрации срезали данные до 2016 года
- В колонку 'Power' заметил аномальные выбросы и по своему анализ сделал срез от 40 до 1001. Так как это странно что у машин 0-39 лошадинные силы
- Так же аномальные выбросы в колонке 'Price' с ними так же выполнили срез

# 2.0 Обучение моделей

In [24]:
# Теперь займемся обучением модели. Сначала подготовим выборки
# Сначала сделаем копию данных и избежим дамми ловушек

avto = auto.copy()
#avto = pd.get_dummies(avto)

In [25]:
# Добавим порядковое кодирование

encoder = OrdinalEncoder()
cat_col = avto.select_dtypes(include=['object']).columns
avto_ordinal = avto.copy()

encoder.fit(avto_ordinal[cat_col])
avto_ordinal[cat_col] = encoder.transform(avto_ordinal[cat_col])

In [26]:
avto_ordinal.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
1,18300,2.0,2011,1.0,190,227.0,125000,2.0,1.0,1.0
2,9800,6.0,2004,0.0,163,117.0,125000,2.0,14.0,0.0
3,1500,5.0,2001,1.0,75,116.0,150000,6.0,38.0,0.0
4,3600,5.0,2008,1.0,69,101.0,90000,2.0,31.0,0.0
5,650,4.0,1995,1.0,102,11.0,150000,6.0,2.0,1.0


In [27]:
# avto_ordinal = pd.get_dummies(avto_ordinal) после порядкового прямое делать 

In [28]:
# Делим выборки на тестовую и обучающую
feature = avto_ordinal.drop(['Price'], axis=1)
target = avto_ordinal['Price']

In [29]:
feature_train, feature_test, target_train, target_test = train_test_split(
    feature, target, test_size=0.25, random_state=12345
)

In [30]:
feature_test, feature_valid, target_test, target_valid = train_test_split(
    feature_test, target_test, test_size=0.25, random_state=12345
)

In [31]:
%%time

# Случайный лес

best_model = None
best_result = 10000
best_est = 0
best_depth = 0

for est in range(1, 7):
    for depth in range(1, 8):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(feature_train, target_train)
        predictions_valid = model.predict(feature_valid)
        result = mean_squared_error(target_valid, predictions_valid) ** 0.5
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

RMSE наилучшей модели на валидационной выборке: 2104.0058944144143 Количество деревьев: 5 Максимальная глубина: 7
CPU times: user 11.8 s, sys: 22.3 ms, total: 11.8 s
Wall time: 11.8 s


In [32]:
# Создам функция расчета метрики RMSE, чтобы не писать в каждой модели самостоятельно
def RMSE(target, predict):
    return (mean_squared_error(target, predict))**0.5
   
RMSE_score = make_scorer(RMSE, greater_is_better=False)

In [33]:
%%time
# LightGBM

model_lgb = lgb.LGBMRegressor()
params_lgb = {
    'max_depth':[25, 50, 75],
    'learning_rate':[0.1, 0.3, 0.5],
    'num_leaves':[500,1200],
    'n_estimators': range(5, 15, 5)
}
grid_lgb = GridSearchCV(model_lgb, params_lgb, scoring=RMSE_score , cv=3)
grid_lgb.fit(feature_train, target_train)

CPU times: user 3min 38s, sys: 13 s, total: 3min 51s
Wall time: 29.5 s


GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.1, 0.3, 0.5],
                         'max_depth': [25, 50, 75],
                         'n_estimators': range(5, 15, 5),
                         'num_leaves': [500, 1200]},
             scoring=make_scorer(RMSE, greater_is_better=False))

In [34]:
%%time
# CatBoost

cat = CatBoostRegressor(loss_function = 'RMSE')
cat_params = {
    'depth': [5, 10],
    'learning_rate': [0.5],
    'iterations': [200]
}
cat_grid = GridSearchCV(cat, cat_params, scoring=RMSE_score, cv=3)
cat_grid.fit(feature_train, target_train)

0:	learn: 3324.1679605	total: 67.9ms	remaining: 13.5s
1:	learn: 2733.1353234	total: 73.6ms	remaining: 7.29s
2:	learn: 2459.5975072	total: 79.4ms	remaining: 5.21s
3:	learn: 2303.7198312	total: 85.2ms	remaining: 4.17s
4:	learn: 2207.9000540	total: 91.1ms	remaining: 3.55s
5:	learn: 2147.3838847	total: 96.9ms	remaining: 3.13s
6:	learn: 2109.8739161	total: 103ms	remaining: 2.83s
7:	learn: 2073.3564769	total: 109ms	remaining: 2.6s
8:	learn: 2040.3244106	total: 114ms	remaining: 2.42s
9:	learn: 2017.2148119	total: 119ms	remaining: 2.25s
10:	learn: 1996.6620490	total: 124ms	remaining: 2.13s
11:	learn: 1977.6286548	total: 131ms	remaining: 2.05s
12:	learn: 1963.9087505	total: 137ms	remaining: 1.97s
13:	learn: 1956.6686290	total: 143ms	remaining: 1.9s
14:	learn: 1935.5499260	total: 148ms	remaining: 1.83s
15:	learn: 1925.5827104	total: 154ms	remaining: 1.77s
16:	learn: 1908.4546988	total: 160ms	remaining: 1.72s
17:	learn: 1898.8769359	total: 165ms	remaining: 1.67s
18:	learn: 1891.1635375	total: 171

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fc61719ae80>,
             param_grid={'depth': [5, 10], 'iterations': [200],
                         'learning_rate': [0.5]},
             scoring=make_scorer(RMSE, greater_is_better=False))

# 3.0 Анализ моделей

In [35]:
# код ревьюера
model = RandomForestRegressor(random_state=12345, n_estimators=5, max_depth=7)

In [36]:
%%time
# время обучения для леса

model.fit(feature_train, target_train)

CPU times: user 652 ms, sys: 2.95 ms, total: 655 ms
Wall time: 653 ms


RandomForestRegressor(max_depth=7, n_estimators=5, random_state=12345)

In [37]:
%%time
# время предсказания для леса

model.predict(feature_valid)

CPU times: user 5.82 ms, sys: 750 µs, total: 6.57 ms
Wall time: 5.68 ms


array([11821.55087061,  1547.21520035,  1908.30596951, ...,
        3378.8474529 ,   981.81055183,  4788.53849471])

In [38]:
# LightGBM
model = lgb.LGBMRegressor(random_state=12345, learning_rate=0.1, max_depth=25, num_leaves=100)

In [39]:
%%time
# время обучения для LightGBM
model.fit(feature_train, target_train)

CPU times: user 4.11 s, sys: 190 ms, total: 4.3 s
Wall time: 575 ms


LGBMRegressor(max_depth=25, num_leaves=100, random_state=12345)

In [40]:
%%time
# время предсказания для LightGBM
model.predict(feature_valid)

CPU times: user 309 ms, sys: 12.4 ms, total: 321 ms
Wall time: 45 ms


array([10390.51648488,  1297.66824029,  1570.20587982, ...,
        4708.76643309,   727.47044632,  5272.10533088])

In [41]:
#CatBoost
model = CatBoostRegressor(learning_rate=0.5, random_state=12345, iterations=100, max_depth=5)

In [42]:
%%time
# время обучения для CatBoost
model.fit(feature_train, target_train)

0:	learn: 3351.0937418	total: 8.61ms	remaining: 853ms
1:	learn: 2747.3127315	total: 16.9ms	remaining: 829ms
2:	learn: 2478.4622668	total: 24.6ms	remaining: 794ms
3:	learn: 2308.4463435	total: 31.6ms	remaining: 757ms
4:	learn: 2204.3037892	total: 39.5ms	remaining: 750ms
5:	learn: 2139.6542684	total: 47.6ms	remaining: 746ms
6:	learn: 2091.7742878	total: 55.5ms	remaining: 738ms
7:	learn: 2067.6471310	total: 62.2ms	remaining: 716ms
8:	learn: 2028.6555658	total: 69.6ms	remaining: 704ms
9:	learn: 2002.3062116	total: 77.1ms	remaining: 694ms
10:	learn: 1987.7535358	total: 85ms	remaining: 688ms
11:	learn: 1960.1991343	total: 92.6ms	remaining: 679ms
12:	learn: 1948.2387102	total: 99.9ms	remaining: 669ms
13:	learn: 1940.1127474	total: 107ms	remaining: 659ms
14:	learn: 1925.9812285	total: 116ms	remaining: 656ms
15:	learn: 1914.4548648	total: 124ms	remaining: 649ms
16:	learn: 1900.0253745	total: 133ms	remaining: 649ms
17:	learn: 1889.1172240	total: 141ms	remaining: 643ms
18:	learn: 1879.8279025	tot

In [43]:
%%time
# время предсказания для CatBoost
model.predict(feature_valid)

CPU times: user 6.93 ms, sys: 1.46 ms, total: 8.4 ms
Wall time: 3.23 ms


array([10173.79193081,  1254.32926995,  1722.44632107, ...,
        5358.68050393,   747.9767075 ,  5455.40698896])

In [44]:
# Построим таблицу для наглядного сравнения значений
index = [
    'RandomForestRegressor',
    'CatBoost',
    'LightGBM'
]
data = {
    'качество предсказания':[
        2104,
        1554,
        302
    ],
    'время обучения модели, ms':[
        677,
        3500,
        5000
    ],
    'время предсказания модели, ms':[
        17,
        7,
        283
    ]
}
final = pd.DataFrame(data=data, index=index)
display(final)

,качество предсказания,"время обучения модели, ms","время предсказания модели, ms"
RandomForestRegressor,2104,677,17
CatBoost,1554,3500,7
LightGBM,302,5000,283


### Мини вывод 3.0
- По анализу моделей, лучший показатель показала модель CatBoost. Данную модель рекомендую заказчику и так же проверим ее в тестовой модели

# 4.0 Тестирование лучшей модели

In [45]:
%%time
# CatBoost
print("CatBoost:",mean_squared_error(target_test,cat_grid.predict(feature_test),squared=False))

CatBoost: 1554.8676501999626
CPU times: user 46.8 ms, sys: 2.8 ms, total: 49.6 ms
Wall time: 19 ms


# 5.0 Вывод

    По итогу проведенной работы я получил:
    
1. Проверил входные данные, выяснил что имеются ошибки. Исправил тип данных, добавил пустые значения в некоторых колонках, удалил дубликаты и не нужные колонки, избавился от аномальных выбросов.
2. Далее после обработки всех данных, я сначала выбрал категориальные колонки и добавил порядковое кодирование для быстрой работы модели. Разбил на тестовую и валидационную выборку и начал строить модели.
3. Я обучал 3 модели: RandomForestRegressor, LGBMRegressor, CatBoostRegressor.
4. После обучения я провел анализ моделей и выяснил время обучения и время предсказания каждой модели.
5. Выяснилось, что модель LGBMRegressor показала самый худший результат, а самый лучший показатель показало CatBoost.

### Мой предикт таков: для поставленной задачи, лучшей моделью для использования решение данной задачи будет CatBoost